**Intoduction**
This is my first kernel at Kaggle. I choosed the Titanic competition which is a good way to introduce feature engineering and ensemble modeling. Firstly, I will display some feature analyses then I'll focus on the feature engineering. Last part concerns modeling and predicting the survival on the Titanic using an voting procedure.

This script follows three main parts:

* Feature analysis
* Feature engineerin
* Modeling

In [76]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'gender_submission.csv', 'test.csv']


2. Loading and Checking data
2.1 Loading the data
Loading the training data set and test data set to variables train_data from the source as given above.

In [77]:
train_data = pd.read_csv("../input/train.csv")
test_data = pd.read_csv("../input/test.csv")

2.2 Joining the Training and test data set. Also storing the indices of the starting values of the data sets in order to divide later.

In [78]:
titanic = train_data.append(test_data,ignore_index=True)
train_index = len(train_data)
test_index = len(titanic) - len(train_data)


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


2.3 Checking the details of every column and it's data

In [79]:
titanic.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450


Checking the number of items in each column and hence predicting the null valued columns, As we see here the total number of entries of the Titanic data set (Train + Test) is 1309. Some columns such as Name, Parch, Pclass,etc. have values for all the 1309 entries but some columns such as Age, Cabin, Embarked, Fare have some null values. Hence we need to deal with these missing values in the respective columns

In [80]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


Extracting the honorifics of each passenger to get a better estimate of their ages. 

In [81]:
titanic['Title'] = titanic.Name.apply(lambda name: name.split(",")[1].split(".")[0].strip() )

Seeing the different types of honorifics and it's frequencies. Since Sir,Don,Jonkheer are roughly similiar so they can be clubbed together under the 'Royalty' category.  Hence we are decreasing the number of titles and normalizing the similiar titles to a single generic title


In [82]:
titanic.Title.value_counts()

Mr              757
Miss            260
Mrs             197
Master           61
Dr                8
Rev               8
Col               4
Major             2
Ms                2
Mlle              2
Sir               1
Capt              1
the Countess      1
Don               1
Lady              1
Dona              1
Jonkheer          1
Mme               1
Name: Title, dtype: int64

In [83]:
normalized_titles = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"
}

The map functions maps the original title to the new generic title as per the above normalized_titles list. 

In [84]:
titanic.Title = titanic.Title.map(normalized_titles)
print(titanic.Title.value_counts())

Mr         757
Miss       262
Mrs        200
Master      61
Officer     23
Royalty      6
Name: Title, dtype: int64


In [85]:
grouped = titanic.groupby(['Sex','Pclass', 'Title'])  
grouped.first()

Age        ...                   Ticket
Sex    Pclass Title                 ...                         
female 1      Miss     58.00        ...                   113783
              Mrs      38.00        ...                 PC 17599
              Officer  49.00        ...                    17465
              Royalty  48.00        ...                    11755
       2      Miss      3.00        ...            SC/Paris 2123
              Mrs      14.00        ...                   237736
       3      Miss     26.00        ...         STON/O2. 3101282
              Mrs      27.00        ...                   347742
male   1      Master    0.92        ...                   113781
              Mr       54.00        ...                    17463
              Officer  44.00        ...                    19928
              Royalty  40.00        ...                 PC 17601
       2      Master    0.83        ...                   248738
              Mr       35.00        ...                   244373
              Officer  42.00        ...                   244310
       3      Master    2.00        ...                   349909
              Mr       22.00        ...                A/5 21171

[17 rows x 10 columns]

Taking median of each category of people and filling the NA values.

In [86]:
grouped.Age.median()
titanic.Age = grouped.Age.apply(lambda x: x.fillna(x.median()))
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 13 columns):
Age            1309 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
Title          1309 non-null object
dtypes: float64(3), int64(4), object(6)
memory usage: 133.0+ KB


In [87]:
titanic.Cabin = titanic.Cabin.fillna('U')
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 13 columns):
Age            1309 non-null float64
Cabin          1309 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
Title          1309 non-null object
dtypes: float64(3), int64(4), object(6)
memory usage: 133.0+ KB


The missing values of embarked are filled with the most occuring value in the embarked column, NA values for Fare are filled with the median value of all Fare prices

In [88]:
most_embarked = titanic.Embarked.value_counts().index[0]
titanic.Embarked = titanic.Embarked.fillna(most_embarked)

titanic.Fare = titanic.Fare.fillna(titanic.Fare.median())

titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 13 columns):
Age            1309 non-null float64
Cabin          1309 non-null object
Embarked       1309 non-null object
Fare           1309 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
Title          1309 non-null object
dtypes: float64(3), int64(4), object(6)
memory usage: 133.0+ KB


Creating a new type called 'FamilySize' which tells us if the passenger had some other relatives on the ship or if they were travelling alone. Hence the +1 indicates the passenger itself

In [89]:
titanic['FamilySize'] = titanic.Parch + titanic.SibSp + 1
titanic.Cabin = titanic.Cabin.map(lambda x: x[0])

In order to covert strings to numbers, mapping 'male' string to 0 and 'female' string to 1

In [90]:
titanic.Sex = titanic.Sex.map({"male": 0, "female":1})
titanic.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title,FamilySize
0,22.0,U,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,0,1,0.0,A/5 21171,Mr,2
1,38.0,C,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,1,1,1.0,PC 17599,Mrs,2
2,26.0,U,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,1,0,1.0,STON/O2. 3101282,Miss,1
3,35.0,C,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,1,1,1.0,113803,Mrs,2
4,35.0,U,S,8.0500,"Allen, Mr. William Henry",0,5,3,0,0,0.0,373450,Mr,1


One Hot encoding for all other types having string values

In [91]:
pclass_dummies = pd.get_dummies(titanic.Pclass, prefix="Pclass")
title_dummies = pd.get_dummies(titanic.Title, prefix="Title")
cabin_dummies = pd.get_dummies(titanic.Cabin, prefix="Cabin")
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix="Embarked")

titanic_dummies = pd.concat([titanic, pclass_dummies, title_dummies, cabin_dummies, embarked_dummies], axis=1)
titanic.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title,FamilySize
0,22.0,U,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,0,1,0.0,A/5 21171,Mr,2
1,38.0,C,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,1,1,1.0,PC 17599,Mrs,2
2,26.0,U,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,1,0,1.0,STON/O2. 3101282,Miss,1
3,35.0,C,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,1,1,1.0,113803,Mrs,2
4,35.0,U,S,8.0500,"Allen, Mr. William Henry",0,5,3,0,0,0.0,373450,Mr,1


Deleting the original columns having string values

In [92]:
titanic_dummies = titanic_dummies.drop(['Name','Ticket','Title','Parch','SibSp','Cabin','Embarked'],axis=1)

In [93]:
titanic_dummies.head()

,Age,Fare,PassengerId,Pclass,Sex,Survived,FamilySize,Pclass_1,Pclass_2,Pclass_3,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,1,3,0,0.0,2,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,38.0,71.2833,2,1,1,1.0,2,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0
2,26.0,7.9250,3,3,1,1.0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,35.0,53.1000,4,1,1,1.0,2,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1
4,35.0,8.0500,5,3,0,0.0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1


Seperating the train and test dataset from titanic_dummies. The train_index is the number of rows of original train data set. Hence all the values prior to it will be in the train set and all values after the train_index will be in test set

In [94]:
train = titanic_dummies[ :train_index]
test = titanic_dummies[train_index: ]

train.Survived = train.Survived.astype(int)


/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [95]:
train.tail()

,Age,Fare,PassengerId,Pclass,Sex,Survived,FamilySize,Pclass_1,Pclass_2,Pclass_3,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Embarked_C,Embarked_Q,Embarked_S
886,27.0,13.00,887,2,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1
887,19.0,30.00,888,1,1,1,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
888,18.0,23.45,889,3,1,0,4,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
889,26.0,30.00,890,1,0,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
890,32.0,7.75,891,3,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0


X contains training data set, Y contains the values for Survived column

In [96]:
X = train.drop('Survived', axis=1).values 
y = train.Survived.values
print(X)

[[ 22.       7.25     1.     ...   0.       0.       1.    ]
 [ 38.      71.2833   2.     ...   1.       0.       0.    ]
 [ 26.       7.925    3.     ...   0.       0.       1.    ]
 ...
 [ 18.      23.45   889.     ...   0.       0.       1.    ]
 [ 26.      30.     890.     ...   1.       0.       0.    ]
 [ 32.       7.75   891.     ...   0.       1.       0.    ]]


In [97]:
X_test = test.drop('Survived', axis=1).values

Deiciding the parameters for the RandomForest CLassifier algorithm

In [98]:
forrest_params = dict(     
    max_depth = [n for n in range(9, 14)],     
    min_samples_split = [n for n in range(4, 11)], 
    min_samples_leaf = [n for n in range(2, 5)],     
    n_estimators = [n for n in range(10, 60, 10)],
)

Import modules and train the dataset

In [99]:
from sklearn.ensemble import RandomForestClassifier
forrest = RandomForestClassifier()

In [100]:
from sklearn.model_selection import GridSearchCV
forest_cv = GridSearchCV(estimator=forrest,     param_grid=forrest_params, cv=5) 
forest_cv.fit(X, y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [9, 10, 11, 12, 13], 'min_samples_split': [4, 5, 6, 7, 8, 9, 10], 'min_samples_leaf': [2, 3, 4], 'n_estimators': [10, 20, 30, 40, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [101]:
print("Best score: {}".format(forest_cv.best_score_))
print("Optimal params: {}".format(forest_cv.best_estimator_))

Best score: 0.8428731762065096
Optimal params: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=12, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=6,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [102]:
forrest_pred = forest_cv.predict(X_test)

Submission

In [103]:
test1 = pd.read_csv("../input/test.csv")
passengerId = test1['PassengerId']
sub2 = pd.DataFrame({'PassengerId': passengerId, 'Survived': forrest_pred})
sub2.to_csv('tnmy_Submission2.csv',index = False)